In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
data = pd.read_csv('data/DS0001/04690-0001-Data.tsv',sep='\t')
data.shape

(3617, 4564)

In [4]:
data.head()

V1  V3  V4    V5  V6  V7  V8         V9   V21  V33  V90  V91  V92  V93  \
0   1   5  28  1986   1   1   1  390400411   381    1    2  -99    1  -99   
1   2   5  28  1986   1   1   2  341202411  2040    1    2  -99    1  -99   
2   3   5  21  1986   1   1   3  341206211  2047    1    2  -99    1  -99   
3   4   5  19  1986   1   1   4          0  5199   51    2  -99    1  -99   
4   5   5  18  1986   5   1   5  390602111  1306    1    2  -99    1  -99   

   V94  V95  V100  V101  V102  V103  V104  V105  V106  V107  V108  V109  V110  \
0  -99  -99     1     1     1     2    69   -99   -99   -99   -99   -99   -99   
1  -99  -99     2     4     1     1    44     2     2    42    11     1    15   
2  -99  -99     3     1     1     1    75   -99   -99   -99   -99   -99   -99   
3  -99  -99     0     2     1     1    25    21     1    45   -99   -99   -99   
4  -99  -99     5     2     1     2    30    11     1     7   -99   -99   -99   

   V111  V112  V113  V114  V115  V116  V117  V118  V119  V120  V121  V122  \
0   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
1    12     2    17   -99   -99   -99   -99   -99   -99   -99   -99   -99   
2   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
3   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
4   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   

   V123  V124  V125  V126  V127  V128  V129  V130  V131  V132  V133  V134  \
0   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
1   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
2   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
3   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
4   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   

   V135  V136  V137  V138  V201  V202  V203  V204  V205  V206  V207  V208  \
0   -99   -99   -99   -99     5     0   -99   -99   -99   -99   -99   -99   
1   -99   -99   -99     5     5     0   -99   -99   -99   -99   -99   -99   
2   -99   -99   -99   -99     1     4     1    55     1    37     1    31   
3   -99   -99   -99     1     5     0   -99   -99   -99   -99   -99   -99   
4   -99   -99   -99     5     5     0   -99   -99   -99   -99   -99   -99   

   V209  V210  V211  V212  V213  V214  V215  V216  V217  V218  V219  V220  \
0   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     0     6   
1   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     0     2   
2     2    45   -99   -99   -99   -99   -99   -99   -99   -99    11     5   
3   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     0     3   
4   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     0     3   

   V221  V222  V223  V224  V225  V301  V302  V303  V304  V305  V306  V307  \
0     2     6     4     4     3     1    40   -99   -99   -99   -99    50   
1     2     3     2     2     2     2    19   -99   -99   -99   -99    96   
2     1     6     1     1     1     2    52   -99   -99   -99   -99    96   
3     2     6     2     3     2     2    52    58    40   -99   -99     8   
4     1     6     2     2     3     2     3    29   -99   -99   -99    33   

   V308  V309  V310  V311  V312  V313  V314  V315  V316  V317  V318  V319  \
0    40   -99   -99   -99     3     2     2     2   -96   -96   -96     2   
1   -99   -99   -99   -99     2     1     4     2     1     2     1     2   
2   -99   -99   -99   -99     2     2     2     3     2     2     2     1   
3   -99   -99   -99   -99     1     1     2     2     1     3     2     4   
4    29    25   -99   -99     1     1     2     2     3     4     3     1   

   V320  V321  V322  V323  V324  V325  V326  V327  V328  V329  V330  V331  \
0     2     3     1     4     4     1     3     2   -96   -96   -96   -96   
1     3     6     1     2     1     2   -99     1     1     4     4     1   
2     3     2     2     1     1     1     4 

In [3]:
print(len(list(data.columns)))

4564


In [6]:
data.describe()

V1           V3           V4      V5           V6  \
count  3617.000000  3617.000000  3617.000000  3617.0  3617.000000   
mean   1809.000000     7.088471    16.135471  1986.0     1.800664   
std    1044.282289     1.191750     8.838172     0.0     1.600719   
min       1.000000     5.000000     1.000000  1986.0     1.000000   
25%     905.000000     6.000000     9.000000  1986.0     1.000000   
50%    1809.000000     7.000000    16.000000  1986.0     1.000000   
75%    2713.000000     8.000000    24.000000  1986.0     1.000000   
max    3617.000000    10.000000    31.000000  1986.0     5.000000   

                V7           V8            V9          V21          V33  \
count  3617.000000  3617.000000  3.617000e+03  3617.000000  3617.000000   
mean      1.175283  1809.000000  3.248283e+08  2407.946641    14.258225   
std       0.818223  1044.282289  2.837840e+08  2165.448814    26.620358   
min       1.000000     1.000000  0.000000e+00     1.000000     1.000000   
25%       1.000000   905.000000  2.540301e+07   905.000000     1.000000   
50%       1.000000  1809.000000  2.907044e+08  1810.000000     1.000000   
75%       1.000000  2713.000000  5.712030e+08  2714.000000     1.000000   
max       5.000000  3617.000000  8.412098e+08  9166.000000    86.000000   

               V90          V91          V92          V93          V94  \
count  3617.000000  3617.000000  3617.000000  3617.000000  3617.000000   
mean      1.818358   -80.660769     1.184130   -51.212054   -80.228090   
std       0.385603    38.935023     0.394013   109.129906    39.897937   
min       1.000000   -99.000000     1.000000   -99.000000   -99.000000   
25%       2.000000   -99.000000     1.000000   -99.000000   -99.000000   
50%       2.000000   -99.000000     1.000000   -99.000000   -99.000000   
75%       2.000000   -99.000000     1.000000   -99.000000   -99.000000   
max       2.000000     4.000000     3.000000   327.000000    22.000000   

               V95         V100         V101    V102         V103  \
count  3617.000000  3617.000000  3617.000000  3617.0  3617.000000   
mean    -79.845729  1421.480232     2.624274     1.0     1.624551   
std      40.877758  1178.949242     1.477560     0.0     0.484306   
min     -99.000000     0.000000     1.000000     1.0     1.000000   
25%     -99.000000   185.000000     2.000000     1.0     1.000000   
50%     -99.000000  1315.000000     2.000000     1.0     2.000000   
75%     -99.000000  2453.000000     4.000000     1.0     2.000000   
max      68.000000  3617.000000    12.000000     1.0     2.000000   

              V104         V105         V106         V107         V108  \
count  3617.000000  3617.000000  3617.000000  3617.000000  3617.000000   
mean     53.459497   -16.189383   -20.887752    13.949959   -52.421620   
std      18.137231    45.564579    41.801663    63.224198    56.724918   
min     -96.000000   -99.000000   -99.000000   -99.000000   -99.000000   
25%      37.000000     2.000000     1.000000     9.000000   -99.000000   
50%      56.000000     2.000000     1.000000    36.000000   -99.000000   
75%      68.000000     2.000000     2.000000    60.000000    11.000000   
max      95.000000    97.000000     2.000000    95.000000    68.000000   

              V109         V110         V111         V112         V113  \
count  3617.000000  3617.000000  3617.000000  3617.000000  3617.000000   
mean    -58.156483   -51.005529   -69.847387   -73.689522   -70.753110   
std      49.368460    58.990854    50.596562    43.628701    49.023017   
min     -99.000000   -99.000000   -99.000000   -99.000000   -99.000000   
25%     -99.000000   -99.000000   -99.000000   -99.000000   -99.000000   
50%     -99.000000   -99.000000   -99.000000   -99.000000   -99.000000   
75%       1.000000    13.000000     2.000000     1.000000     1.000000   
max       2.000000    95.000000    67.000000     2.000000    81.000000   

              V114         V115         V116         V117         V118  \
count  3617.000000  3

- Originally 3617 respondents
- Listwise deletion to 1640 participants
- Use dropna() function for Listwise deletion
- All of these participants had a spouse, a child, and friends when responding to the survey

- A1: Children = 2
- C7: Martial status = V410
- 

- V138: A1. Children in HH -> 1 No Child in HH && V201
- V401: C1. Marital Status -> 5 Never Marry
- V221: A3. Visit Friends -> 6 Never

In [29]:
data.V138.value_counts()

 1     1562
 5     1250
-99     805
Name: V138, dtype: int64

In [44]:
# Married, have friends

sample = data[(data['V401'] == 1) & (data['V221'] != 6) & (data['V221'] != -96)]

In [48]:
no_child = sample[(sample['V138'] == 1) & (sample['V201'] == 5)]
print(no_child.shape)

(176, 4564)


In [49]:
df = pd.concat([sample, no_child, no_child]).drop_duplicates(keep=False)
print(df.shape)

(1718, 4564)


In [50]:
df.V222.value_counts()

 6     606
 4     279
 1     233
 3     215
 2     197
 5     187
-96      1
Name: V222, dtype: int64